### Ensemble Retriever (하이브리드 검색)
- 여러 검색 방식을 결합하여 검색 품질을 향상시키는 기법

- 일반적인 조합은 **BM25 키워드 검색**과 **벡터 유사도 검색**을 결합하는 하이브리드 검색

In [1]:
from langchain_core.documents import Document

# LangChain 관련 기술 문서 샘플
documents = [
    Document(
        page_content="LangChain은 대규모 언어 모델(LLM)을 활용한 애플리케이션 개발 프레임워크입니다. "
                     "체인, 에이전트, 메모리 등의 핵심 개념을 제공합니다.",
        metadata={"source": "langchain_intro", "category": "overview"}
    ),
    Document(
        page_content="RAG(Retrieval-Augmented Generation)는 검색 증강 생성 기법으로, "
                     "외부 문서를 검색하여 LLM의 응답 품질을 향상시킵니다.",
        metadata={"source": "rag_intro", "category": "rag"}
    ),
    Document(
        page_content="벡터 데이터베이스는 임베딩 벡터를 저장하고 유사도 검색을 수행합니다. "
                     "FAISS, Chroma, Pinecone 등이 대표적입니다.",
        metadata={"source": "vectordb", "category": "database"}
    ),
    Document(
        page_content="BM25는 키워드 기반 검색 알고리즘으로, TF-IDF를 개선한 방식입니다. "
                     "정확한 키워드 매칭에 강점이 있습니다.",
        metadata={"source": "bm25", "category": "search"}
    ),
    Document(
        page_content="하이브리드 검색은 키워드 검색과 시맨틱 검색을 결합합니다. "
                     "BM25와 벡터 검색의 장점을 모두 활용할 수 있습니다.",
        metadata={"source": "hybrid", "category": "search"}
    ),
    Document(
        page_content="임베딩 모델은 텍스트를 벡터로 변환합니다. "
                     "OpenAI, HuggingFace, Cohere 등에서 다양한 모델을 제공합니다.",
        metadata={"source": "embedding", "category": "model"}
    ),
]

print(f"총 {len(documents)}개의 문서 준비 완료")

총 6개의 문서 준비 완료


In [4]:
from langchain_community.retrievers import BM25Retriever

# BM25 검색기 생성
bm25_retriever = BM25Retriever.from_documents(
    documents=documents,
    k=3  # 상위 3개 문서 반환
)

# BM25 검색 테스트
query = "BM25 알고리즘의 특징"
bm25_results = bm25_retriever.invoke(query)

print("=== BM25 검색 결과 ===")
for i, doc in enumerate(bm25_results, 1):
    print(f"{i}. [{doc.metadata['source']}] {doc.page_content[:50]}...")

=== BM25 검색 결과 ===
1. [embedding] 임베딩 모델은 텍스트를 벡터로 변환합니다. OpenAI, HuggingFace, Coher...
2. [hybrid] 하이브리드 검색은 키워드 검색과 시맨틱 검색을 결합합니다. BM25와 벡터 검색의 장점을 ...
3. [bm25] BM25는 키워드 기반 검색 알고리즘으로, TF-IDF를 개선한 방식입니다. 정확한 키워드...


**한국어 토크나이저 설정 (kiwipiepy)**

BM25는 기본적으로 공백 기반 토큰화를 사용하는데, 한국어는 교착어로 형태소 분석이 필요합니다.

kiwipiepy를 사용하면 한국어 검색 성능을 크게 향상시킬 수 있습니다.

In [5]:
from kiwipiepy import Kiwi

# Kiwi 형태소 분석기 초기화
kiwi = Kiwi()

# 형태소 분석 테스트
text = "LangChain은 대규모 언어 모델을 활용한 애플리케이션 개발 프레임워크입니다."
tokens = kiwi.tokenize(text)

print("=== Kiwi 형태소 분석 결과 ===")
for token in tokens:
    print(f"  {token.form:10} | {token.tag:5} | 위치: {token.start}-{token.start + token.len}")

=== Kiwi 형태소 분석 결과 ===
  LangChain  | SL    | 위치: 0-9
  은          | JX    | 위치: 9-10
  대          | XPN   | 위치: 11-12
  규모         | NNG   | 위치: 12-14
  언어         | NNG   | 위치: 15-17
  모델         | NNG   | 위치: 18-20
  을          | JKO   | 위치: 20-21
  활용         | NNG   | 위치: 22-24
  하          | XSV   | 위치: 24-25
  ᆫ          | ETM   | 위치: 24-25
  애플리케이션     | NNG   | 위치: 26-32
  개발         | NNG   | 위치: 33-35
  프레임        | NNG   | 위치: 36-39
  워크         | NNG   | 위치: 39-41
  이          | VCP   | 위치: 41-42
  ᆸ니다        | EF    | 위치: 41-44
  .          | SF    | 위치: 44-45


In [6]:
from kiwipiepy import Kiwi
from langchain_community.retrievers import BM25Retriever

# Kiwi 초기화
kiwi = Kiwi()

def kiwi_tokenize(text: str) -> list[str]:
    """Kiwi를 사용한 한국어 토크나이저"""
    tokens = kiwi.tokenize(text)
    # 형태소(form)만 추출하여 리스트로 반환
    return [token.form for token in tokens]

# 테스트
print("토큰화 결과:", kiwi_tokenize("금융보험은 장기적인 자산 관리 상품입니다."))

토큰화 결과: ['금융', '보험', '은', '장기', '적', '이', 'ᆫ', '자산', '관리', '상품', '이', 'ᆸ니다', '.']


In [7]:
# Kiwi 토크나이저를 적용한 BM25 Retriever 생성
bm25_retriever_kiwi = bm25_retriever.from_documents(
    documents,
    k=3,
    preprocess_func=kiwi_tokenize
)

# 검색 테스트
query = "언어 모델 프레임워크"
results = bm25_retriever_kiwi.invoke(query)

print("=== Kiwi 적용 BM25 검색 결과 ===")
for i, doc in enumerate(results, 1):
    print(f"{i}. [{doc.metadata['source']}] {doc.page_content[:50]}...")

=== Kiwi 적용 BM25 검색 결과 ===
1. [langchain_intro] LangChain은 대규모 언어 모델(LLM)을 활용한 애플리케이션 개발 프레임워크입니다....
2. [embedding] 임베딩 모델은 텍스트를 벡터로 변환합니다. OpenAI, HuggingFace, Coher...
3. [hybrid] 하이브리드 검색은 키워드 검색과 시맨틱 검색을 결합합니다. BM25와 벡터 검색의 장점을 ...


검색 품질을 높이려면 명사만 추출하는 것이 효과적일 수 있습니다

In [8]:
def kiwi_tokenize_nouns(text: str) -> list[str]:
    """명사(NNG, NNP)만 추출하는 토크나이저"""
    tokens = kiwi.tokenize(text)
    # 일반명사(NNG), 고유명사(NNP)만 추출
    return [token.form for token in tokens if token.tag in ("NNG", "NNP")]

# 테스트
text = "LangChain은 대규모 언어 모델을 활용한 애플리케이션 개발 프레임워크입니다."
print("전체 형태소:", kiwi_tokenize(text))
print("명사만 추출:", kiwi_tokenize_nouns(text))

전체 형태소: ['LangChain', '은', '대', '규모', '언어', '모델', '을', '활용', '하', 'ᆫ', '애플리케이션', '개발', '프레임', '워크', '이', 'ᆸ니다', '.']
명사만 추출: ['규모', '언어', '모델', '활용', '애플리케이션', '개발', '프레임', '워크']


In [9]:
# 검색 테스트
query = "BM25 알고리즘의 특징"
results = bm25_retriever_kiwi.invoke(query)

print("=== Kiwi 적용 BM25 검색 결과 ===")
for i, doc in enumerate(results, 1):
    print(f"{i}. [{doc.metadata['source']}] {doc.page_content[:50]}...")

=== Kiwi 적용 BM25 검색 결과 ===
1. [bm25] BM25는 키워드 기반 검색 알고리즘으로, TF-IDF를 개선한 방식입니다. 정확한 키워드...
2. [hybrid] 하이브리드 검색은 키워드 검색과 시맨틱 검색을 결합합니다. BM25와 벡터 검색의 장점을 ...
3. [embedding] 임베딩 모델은 텍스트를 벡터로 변환합니다. OpenAI, HuggingFace, Coher...


In [13]:
# 기본 BM25 (공백 기반)
bm25_default = BM25Retriever.from_documents(documents, k=3)

# Kiwi 적용 BM25
bm25_kiwi = BM25Retriever.from_documents(
    documents, 
    k=3, 
    preprocess_func=kiwi_tokenize
)

def compare_bm25(query: str):
    """기본 BM25와 Kiwi BM25 비교"""
    print(f"쿼리: '{query}'\n")

    print("=== 기본 BM25 ===")
    for i, doc in enumerate(bm25_default.invoke(query)[:2], 1):
        print(f"  {i}. {doc.metadata['source']}")

    print("\n=== Kiwi BM25 ===")
    for i, doc in enumerate(bm25_kiwi.invoke(query)[:2], 1):
        print(f"  {i}. {doc.metadata['source']}")

compare_bm25("BM25 알고리즘의 특징")

쿼리: 'BM25 알고리즘의 특징'

=== 기본 BM25 ===
  1. embedding
  2. hybrid

=== Kiwi BM25 ===
  1. bm25
  2. hybrid


In [14]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

# 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# FAISS 벡터스토어 생성
vectorstore = FAISS.from_documents(documents, embeddings)

# 벡터 검색기 생성
vector_retriever = vectorstore.as_retriever(search_kwargs={'k': 3})

# 벡터 검색 테스트
vector_results = vector_retriever.invoke(query)

print("=== 벡터 검색 결과 ===")
for i, doc in enumerate(vector_results, 1):
    print(f"{i}. [{doc.metadata['source']}] {doc.page_content[:50]}...")

e:\Portfolio\Study\hello-langchain\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


=== 벡터 검색 결과 ===
1. [bm25] BM25는 키워드 기반 검색 알고리즘으로, TF-IDF를 개선한 방식입니다. 정확한 키워드...
2. [hybrid] 하이브리드 검색은 키워드 검색과 시맨틱 검색을 결합합니다. BM25와 벡터 검색의 장점을 ...
3. [rag_intro] RAG(Retrieval-Augmented Generation)는 검색 증강 생성 기법으로...


**FAISS와 BF25 앙상블**

In [15]:
from langchain_classic.retrievers import EnsembleRetriever

# Kiwi 토크나이저를 적용한 BM25와 벡터 검색 결합
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_kiwi, vector_retriever],
    weights=[0.5, 0.5]
)

# Ensemble 검색 테스트
query = "BM25 알고리즘의 특징"
ensemble_results = ensemble_retriever.invoke(query)

print("=== Ensemble 검색 결과 ===")
for i, doc in enumerate(ensemble_results, 1):
    print(f"{i}. [{doc.metadata['source']}] {doc.page_content[:50]}...")

=== Ensemble 검색 결과 ===
1. [bm25] BM25는 키워드 기반 검색 알고리즘으로, TF-IDF를 개선한 방식입니다. 정확한 키워드...
2. [hybrid] 하이브리드 검색은 키워드 검색과 시맨틱 검색을 결합합니다. BM25와 벡터 검색의 장점을 ...
3. [embedding] 임베딩 모델은 텍스트를 벡터로 변환합니다. OpenAI, HuggingFace, Coher...
4. [rag_intro] RAG(Retrieval-Augmented Generation)는 검색 증강 생성 기법으로...


In [16]:
# 다양한 가중치 조합 테스트
weight_configs = [
    ([0.7, 0.3], "BM25 우선"),      # 키워드 매칭 중시
    ([0.5, 0.5], "균형"),           # 균형 잡힌 접근
    ([0.3, 0.7], "벡터 우선"),      # 의미적 유사도 중시
]

query = "대규모 언어 모델 애플리케이션 개발"

print(f"쿼리: '{query}'\n")

for weights, description in weight_configs:
    ensemble = EnsembleRetriever(
        retrievers=[bm25_kiwi, vector_retriever],
        weights=weights
    )
    results = ensemble.invoke(query)

    print(f"=== {description} (BM25:{weights[0]}, Vector:{weights[1]}) ===")
    for i, doc in enumerate(results[:3], 1):
        print(f"  {i}. [{doc.metadata['source']}]")
    print()


쿼리: '대규모 언어 모델 애플리케이션 개발'

=== BM25 우선 (BM25:0.7, Vector:0.3) ===
  1. [langchain_intro]
  2. [embedding]
  3. [hybrid]

=== 균형 (BM25:0.5, Vector:0.5) ===
  1. [langchain_intro]
  2. [embedding]
  3. [hybrid]

=== 벡터 우선 (BM25:0.3, Vector:0.7) ===
  1. [langchain_intro]
  2. [embedding]
  3. [rag_intro]



In [17]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# LLM 초기화
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 프롬프트 템플릿
template = """다음 컨텍스트를 기반으로 질문에 답변하세요.

컨텍스트:
{context}

질문: {question}

답변:"""

prompt = ChatPromptTemplate.from_template(template)

# 문서 포맷팅 함수
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# RAG 체인 구성
rag_chain = (
    {"context": ensemble_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 질의응답 실행
question = "하이브리드 검색의 장점은 무엇인가요?"
answer = rag_chain.invoke(question)

print(f"질문: {question}")
print(f"답변: {answer}")

질문: 하이브리드 검색의 장점은 무엇인가요?
답변: 하이브리드 검색의 장점은 키워드 검색과 시맨틱 검색의 장점을 모두 활용할 수 있다는 점입니다. 이를 통해 사용자는 정확한 키워드 매칭을 통해 필요한 정보를 빠르게 찾을 수 있으며, 동시에 시맨틱 검색을 통해 더 넓은 의미의 관련 정보를 탐색할 수 있습니다. 이로 인해 검색 결과의 품질과 다양성이 향상되며, 사용자의 의도에 맞는 보다 정확하고 유용한 정보를 제공할 수 있습니다. 또한, BM25와 벡터 검색의 조합으로 인해 검색의 정확성과 유연성이 모두 강화됩니다.
